In [680]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, assemble, transpile
import numpy as np
import math
from qiskit.visualization import plot_histogram, plot_bloch_multivector, array_to_latex
from qiskit.ignis.verification import marginal_counts
from threading import Thread, Lock, Condition

In [681]:
num_players = 2
tosses = pow(num_players, 2)
k = 3*np.log2(num_players)
num_qubits = math.ceil(k)
upper_bound = pow(num_players,3)

lock = Lock()

c_messages = []
l_messages = []

c_measurements = []
l_measurements = []

In [682]:
def coin_toss(player_id, condition):  

    coin = QuantumCircuit(num_players)

    coin.h(0)

    for qubit in range(1,num_players):
        coin.cx(0,qubit)

    # coin.barrier()

    lock.acquire()
    try:
        for receiver in range(num_players):
            coin_results = [player_id, receiver, coin]
            
            c_messages.append(coin_results)
            # print(f'Player {player_id} tossed coin for player {receiver}')
    finally:
        lock.release()   

    with condition:
        if(len(c_messages) == tosses):
            condition.notify_all()

    return coin

In [683]:
def measure_coins(reader_id, owner_id):
    backend = Aer.get_backend('aer_simulator')

    lock.acquire()
    
    for i in range(len(c_messages)):
        if reader_id == c_messages[i][1] & owner_id == c_messages[i][0]:

            c_messages[i][2].measure_all()

            print(c_messages[i][2])

            aer_sim = Aer.get_backend('aer_simulator')
            qobj = assemble(c_messages[i][2], shots=1, memory=True)
            result = aer_sim.run(qobj).result()

            memory = result.get_memory()[0][0:num_players]

            c_values = [reader_id, c_messages[i][0], memory]
            c_measurements.append(c_values)

            # print(f"Reader {reader_id} decides {memory} in player {c_messages[i][0]} circuit")
            print(f"Player {reader_id} decides {memory}")
            
    lock.release()

    return c_measurements
               
    

In [684]:
def leader_toss(player_id, condition):
    
    l_registers = []

    for a in range(num_players):
        q_name = 'q' + str(a)
        q_reg = QuantumRegister(num_qubits, q_name)
        l_registers.append(q_reg)

    leader = QuantumCircuit(*l_registers)

    cr = ClassicalRegister(num_players*num_qubits, 'c')

    leader.add_register(cr)

    for qubit in range(num_qubits):
        leader.h(l_registers[0][qubit])
    for a in range(1,num_players):
        for qubit in range(num_qubits):
            leader.cx(l_registers[0][qubit], l_registers[a][qubit])

    leader.barrier()   

    lock.acquire()
    try:
        for receiver in range(num_players):
            leader_results = [player_id, receiver, leader]

            l_messages.append(leader_results)
            # print(f'Player {player_id} chose leader for player {receiver}')
    finally:
        lock.release()

    with condition:
        if(len(l_messages) == tosses):
            condition.notify_all()   

    return leader
    

In [685]:
def measure_leader(reader_id, condition):
    backend = Aer.get_backend('aer_simulator')

    lock.acquire()
    for i in range(len(l_messages)):
        if reader_id == l_messages[i][1]:
            # start_id = reader_id * num_qubits

            for qubit in range(num_qubits*num_players):
                l_messages[i][2].measure(qubit, qubit)                 
            l_messages[i][2].barrier()
            print(l_messages[i][2])

            aer_sim = Aer.get_backend('aer_simulator')
            qobj = assemble(l_messages[i][2], shots=1, memory=True)
            result = aer_sim.run(qobj).result()

            memory = result.get_memory()[0][0:num_qubits]
            l_value = [reader_id, l_messages[i][0], memory]
            l_measurements.append(l_value)

            print(f"Player {reader_id} decides {memory} in player {l_messages[i][0]} circuit")

            
    lock.release()
    
    with condition:
        if(len(l_measurements) == tosses):
            condition.notify_all()  

    return l_measurements

    

In [686]:
def get_highest_leader(reader_id):
    numbers = []
    for i in range(len(l_measurements)):
        if(l_measurements[i][0] == reader_id):
            numbers.append([l_measurements[i][1], l_measurements[i][2]])

    leader_coin, measured_leader = max(numbers, key=lambda x: x[1])
    print(measured_leader)
    max_value = int(measured_leader,2)
    print(f"Max Value: {max_value} From player: {leader_coin}")

    max_count = 1
    for n in range(len(numbers)):
        if(numbers[n][1] == max_value):
            max_count += 1

    if(max_count > 1):
        leader_coin = 1


    return leader_coin

In [687]:
# l_cond = Condition()
# leader = leader_toss(0, l_cond)
# # leader.draw()
# l = measure_leader(0)
# print(l_measurements[l][2])

# l_messages[0][2].barrier()
# for qubit in range(num_qubits):
#     l_messages[0][2].x(qubit).c_if(qubit, 1)
#     l_messages[0][2].z(qubit).c_if(qubit, 1)
    
# print(l_messages[0][2])

# cr_result = ClassicalRegister(num_qubits)
# l_messages[0][2].add_register(cr_result)
# for qubit in range(num_qubits):
#     l_messages[0][2].measure(qubit, qubit)  
# print(l_messages[0][2])


In [688]:
def fail_stop_protocol(player_id, c_condition, l_condition, m_cond):
    
    coin = coin_toss(player_id, c_condition)
    with c_condition:
        while(len(c_messages) != tosses):
            c_condition.wait()

    leader = leader_toss(player_id, l_condition)
    with l_condition:
        while(len(l_messages) != tosses):
            l_condition.wait()
    
    leader_measurements = measure_leader(player_id, m_cond)

    with l_condition:
        while(len(l_messages) != tosses):
            l_condition.wait()

    leader_id = get_highest_leader(player_id)

    coin_measurements = measure_coins(player_id, leader_id)


In [689]:
threads = []
c_cond = Condition()
l_cond = Condition()
m_cond = Condition()
for i in range(num_players):
    t = Thread(target=fail_stop_protocol, args=(i,c_cond,l_cond,m_cond,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

      ┌───┐                ░ ┌─┐                ░ 
q0_0: ┤ H ├──■─────────────░─┤M├────────────────░─
      ├───┤  │             ░ └╥┘┌─┐             ░ 
q0_1: ┤ H ├──┼────■────────░──╫─┤M├─────────────░─
      ├───┤  │    │        ░  ║ └╥┘┌─┐          ░ 
q0_2: ┤ H ├──┼────┼────■───░──╫──╫─┤M├──────────░─
      └───┘┌─┴─┐  │    │   ░  ║  ║ └╥┘┌─┐       ░ 
q1_0: ─────┤ X ├──┼────┼───░──╫──╫──╫─┤M├───────░─
           └───┘┌─┴─┐  │   ░  ║  ║  ║ └╥┘┌─┐    ░ 
q1_1: ──────────┤ X ├──┼───░──╫──╫──╫──╫─┤M├────░─
                └───┘┌─┴─┐ ░  ║  ║  ║  ║ └╥┘┌─┐ ░ 
q1_2: ───────────────┤ X ├─░──╫──╫──╫──╫──╫─┤M├─░─
                     └───┘ ░  ║  ║  ║  ║  ║ └╥┘ ░ 
 c: 6/════════════════════════╩══╩══╩══╩══╩══╩════
                              0  1  2  3  4  5    
Player 0 decides 010 in player 1 circuit
      ┌───┐                ░ ┌─┐                ░ 
q0_0: ┤ H ├──■─────────────░─┤M├────────────────░─
      ├───┤  │             ░ └╥┘┌─┐             ░ 
q0_1: ┤ H ├──┼────■────────░──╫─┤M├──────